In [1]:
import os
from os.path import join, exists, basename
import argparse
import numpy as np
import paddle.fluid as fluid
import paddle
paddle.seed(0)
import random
np.random.seed(42) 
random.seed(42)
fluid.default_startup_program().random_seed = 42
fluid.default_main_program().random_seed = 42
import paddle.nn as nn
import pgl

from pahelix.model_zoo.gem_model import GeoGNNModel
from pahelix.utils import load_json_config
from pahelix.datasets.inmemory_dataset import InMemoryDataset

from src.model import DownstreamModel
from src.featurizer import DownstreamTransformFn, DownstreamCollateFn
from src.utils import get_dataset, create_splitter, get_downstream_task_names, get_dataset_stat, \
        calc_rocauc_score, calc_rmse, calc_mae, exempt_parameters


In [ ]:
def feature_extract_regr(args, model,dataset, collate_fn, metric):
    data_gen = dataset.get_data_loader(
            batch_size=args.batch_size, 
            num_workers=args.num_workers, 
            shuffle=False,
            collate_fn=collate_fn)
    total_features=[]
    total_label = []
    smiles_list=[]
    model.eval()
    for atom_bond_graphs, bond_angle_graphs, labels,smiles,atom_poses in data_gen:
        atom_bond_graphs = atom_bond_graphs.tensor()
        bond_angle_graphs = bond_angle_graphs.tensor()
        labels = paddle.to_tensor(labels, 'float32')
        features = model(atom_bond_graphs, bond_angle_graphs)
        smiles_list.extend(smiles)
        total_features.append(total_features)
        total_label.append(labels.numpy())
    total_features = np.concatenate(features, 0)
    total_label = np.concatenate(total_label, 0)
    final_dic={"smiles":[],"features":[]}
    task_names = get_downstream_task_names(args.dataset_name, args.data_path)
    for task_name in task_names:
        final_dic["lbl_"+task_name]=[]
    for i in range(total_label.shape[0]):
        final_dic["smiles"].append(smiles_list[i])
        final_dic["features"].append(total_features[i])
        for j in range(len(total_label[i])):
            final_dic["lbl_"+task_names[j]].append(total_label[i][j].item())
    import pickle
    with open(f'./{args.model_dir}/{args.dataset_name}-{args.mode}-seed={args.seed}.pickle', 'wb') as handle:
        pickle.dump(final_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
